In [3]:
!pip install tensorflow

  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 MB 4.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 2.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 8.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 7.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 8.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 7.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 5.4 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.8 MB/s et

In [4]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

import matplotlib.pyplot as plt

2024-08-05 22:24:33.505641: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Function to prepare data for LSTM with item_name encoding
def prepare_data(df, feature_columns, target_column, time_steps):
    # One-hot encode item_name
    item_encoder = OneHotEncoder(sparse=False)
    item_encoded = item_encoder.fit_transform(df[['item_name']])
    item_names = item_encoder.categories_[0]
    
    data = df[feature_columns].values
    target = df[target_column].values.reshape(-1, 1)

    # Concatenate item_name encoding with other features
    data_combined = np.hstack((item_encoded, data))

    scaler_data = MinMaxScaler()
    scaler_target = MinMaxScaler()

    data_scaled = scaler_data.fit_transform(data_combined)
    target_scaled = scaler_target.fit_transform(target)

    X, y = [], []
    for i in range(time_steps, len(data_scaled)):
        X.append(data_scaled[i - time_steps:i])
        y.append(target_scaled[i])
    X, y = np.array(X), np.array(y)

    return X, y, scaler_target, item_names


# Function to create LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Prepare the data

In [6]:
df = pd.read_csv("./train_data.csv")

In [7]:
feature_columns = ['avg_bottle_price', 'total_volume_sold_liters', 'total_sale_dollars',
                   'avg_bottle_profit', 'day_of_week', 'week_of_year', 'month', 'year',
                   'ma7_total_amount_sold', 'ma7_avg_bottle_price', 'ma7_total_sale_dollars',
                   'ma3_total_amount_sold', 'ma3_avg_bottle_price', 'ma3_total_sale_dollars']
target_column = 'total_amount_sold'
time_steps = 7

X, y, scaler_target, item_names = prepare_data(df, feature_columns, target_column, time_steps)

# Split the data into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

/opt/anaconda3/envs/bunge/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Create and train the LSTM model

In [8]:
model = create_lstm_model((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/50
14013/14013 [==============================] - 135s 9ms/step - loss: 1.1904e-04 - val_loss: 1.4056e-04
Epoch 2/50
14013/14013 [==============================] - 169s 12ms/step - loss: 9.2348e-05 - val_loss: 1.2394e-04
Epoch 3/50
14013/14013 [==============================] - 154s 11ms/step - loss: 8.9040e-05 - val_loss: 2.2160e-04
Epoch 4/50
14013/14013 [==============================] - 158s 11ms/step - loss: 8.6657e-05 - val_loss: 1.7915e-04
Epoch 5/50
14013/14013 [==============================] - 227s 16ms/step - loss: 8.3576e-05 - val_loss: 1.4100e-04
Epoch 6/50
14013/14013 [==============================] - 218s 16ms/step - loss: 8.1979e-05 - val_loss: 1.8187e-04
Epoch 7/50
14013/14013 [==============================] - 144s 10ms/step - loss: 7.9617e-05 - val_loss: 2.0989e-04
Epoch 8/50
14013/14013 [==============================] - 167s 12ms/step - loss: 8.0381e-05 - val_loss: 2.3413e-04
Epoch 9/50
14013/14013 [==============================] - 141s 10ms/step - loss: 

## Make predictions

In [ ]:
y_pred_scaled = model.predict(X_test)
y_pred = scaler_target.inverse_transform(y_pred_scaled)
y_test_actual = scaler_target.inverse_transform(y_test)

## Evaluate the model

In [ ]:
rmse = np.sqrt(np.mean((y_pred - y_test_actual) ** 2))
mae = np.mean(np.abs(y_pred - y_test_actual))
print(f'RMSE: {rmse:.2f}, MAE: {mae:.2f}')

## Visualize the results

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(y_test_actual, color='blue', label='Actual Total Amount Sold')
plt.plot(y_pred, color='red', label='Predicted Total Amount Sold')
plt.title('LSTM Model - Total Amount Sold Prediction')
plt.xlabel('Time')
plt.ylabel('Total Amount Sold')
plt.legend()
plt.show()